## 卷积神经网络用于自然语言处理示例

In [1]:
# 示例代码运行环境
%load_ext watermark
%watermark -p tensorflow,numpy -v -m

CPython 2.7.6
IPython 5.1.0

tensorflow 1.0.1
numpy 1.12.1

compiler   : GCC 4.8.4
system     : Linux
release    : 4.4.0-21-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
import tensorflow as tf

In [3]:
vocab_size = 80000
word_embed_size = 128

In [4]:
tf.reset_default_graph()

随机生成词向量

In [5]:
W = tf.Variable(
                tf.random_uniform([vocab_size, word_embed_size], -1.0, 1.0),
                name="W")

根据句子的 ID 查找词向量

In [6]:
embeds = tf.nn.embedding_lookup(W, [[0, 1, 2, 3]])

In [9]:
# 句子长度
sentence_length = 4

In [10]:
# 注意观察 embedding 维度。这里只有一个样本
print(embeds)

Tensor("embedding_lookup:0", shape=(1, 4, 128), dtype=float32)


In [11]:
# 自行查看 expand_dims 的 API 说明。这里是为了适应 conv2d 等参数，拓展了一个维度 (in_channel)，长度是 1
embeds_expand = tf.expand_dims(embeds, -1)

In [13]:
# expand_dims 功能演示
with tf.Session() as sess:
    c = tf.constant([1, 2, 3])
    print(c.eval())
    print(tf.expand_dims(c, 0).eval())
    print(tf.expand_dims(c, -1).eval())

[1 2 3]
[[1 2 3]]
[[1]
 [2]
 [3]]


In [13]:
# 扩展之后的维度
print(embeds_expand)

Tensor("ExpandDims:0", shape=(1, 4, 128, 1), dtype=float32)


In [14]:
# 扩展之后 eval 结果的维度 （和上面一样，但是这里出来是 numpy 的 shape)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(embeds.eval().shape)
    print(embeds_expand.eval().shape)

(1, 4, 128)
(1, 4, 128, 1)


In [15]:
# max_pool
with tf.name_scope("conv-maxpool"):
    filter_num = 64
    window_size = 3
    filter_shape = [window_size, word_embed_size, 1, filter_num]
    # W 和 b 是卷积的参数
    W = tf.Variable(tf.random_uniform(filter_shape, -1.0, 1.0), name="W")
    # bias 和 filter_num 个数是一样的
    b = tf.Variable(tf.constant(0.0, shape=[filter_num]), name="b")
    # 步长为1，这里不做 Padding，因此句子太短的话可能要丢掉。可自行尝试加 padding（不加也不影响作业评分）
    conv = tf.nn.conv2d(
                    embeds_expand,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
    # 卷积出来的结果加上 bias
    conv_hidden = tf.nn.tanh(tf.add(conv, b), name="tanh")

    # 因为没有 padding，出来的结果个数是 sequence_length - window_size + 1，如果加了 padding 这里要对应更改。
    pool = tf.nn.max_pool(
                    conv_hidden,
                    ksize=[1, sentence_length - window_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")

目前 tensorflow 还不支持动态 max_pool size，所以 ksize 只能用常数固定，

因为不同句子 sequence_length 不一样，因此目前这里目前还没法做到处理边长句子。
    
一个解决方案是用人工 Padding 的方式，根据语料中最长的句子的长度来扩展所有句子，归一化到统一的长度。即所有句子都通过 Padding 一个特殊符号的方式，扩展为固定长度。

**注意这个是 Tensorflow 目前的限制**，用其他一些支持动态 max_pool 的库不需要 padding。事实上这也会造成计算量的浪费。

鼓励大家多看中间结果的维度，加深理解

In [16]:
print(conv)
print(conv_hidden)
print(pool)

Tensor("conv-maxpool/conv:0", shape=(1, 2, 1, 64), dtype=float32)
Tensor("conv-maxpool/tanh:0", shape=(1, 2, 1, 64), dtype=float32)
Tensor("conv-maxpool/pool:0", shape=(1, 1, 1, 64), dtype=float32)


卷积 + max pooling 之后的结果可以再接 dense layer (全连接层）

根据这个框架改成符合作业要求的脚本，用于情感分类